In [2]:
import pandas as pd
import glob

In [3]:
# Carica il CSV in un DataFrame
df = pd.read_csv('dataset/flights.csv')

In [4]:
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2009-01-01,WN,2497,TUS,LAS,1100.0,1056.0,-4.0,6.0,1102.0,...,80.0,67.0,59.0,365.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2009-01-01,DL,1590,ATL,BHM,1555.0,1602.0,7.0,13.0,1615.0,...,53.0,46.0,28.0,134.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2009-01-01,US,1968,SJU,BOS,1445.0,1439.0,-6.0,20.0,1459.0,...,245.0,253.0,229.0,1674.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2009-01-01,CO,559,IAH,SNA,925.0,925.0,0.0,13.0,938.0,...,221.0,204.0,187.0,1347.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2009-01-01,FL,373,CMH,MCO,804.0,759.0,-5.0,10.0,809.0,...,136.0,130.0,113.0,802.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
len(df)

119271

In [4]:
print(df.columns)
print(len(df.columns))

Index(['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF',
       'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY',
       'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'Unnamed: 27'],
      dtype='object')
28


# ETL

### Definizione chiavi

In [5]:
# Conta e stampa il numero di duplicati
num_duplicati = df.duplicated().sum()
print(f"Il dataset contiene {num_duplicati} duplicati.")

Il dataset contiene 0 duplicati.


In [6]:
# Oppure, per modificare il DataFrame originale direttamente, puoi usare inplace=True
df.drop_duplicates(inplace=True)

# Controlla se ci sono ancora duplicati
if not df.duplicated().any():
    print("Tutti i duplicati sono stati rimossi.")

Tutti i duplicati sono stati rimossi.


In [7]:
print(f"Il dataset contiene {len(df)} elementi")

Il dataset contiene 119271 elementi


In [8]:
candidate_columns = [
    'FL_DATE',
    'OP_CARRIER',
    'OP_CARRIER_FL_NUM',
    'ORIGIN',
    'DEST'
]

if not df[candidate_columns].duplicated().any():
    print(f"La colonna '{candidate_columns}' potrebbe essere una chiave primaria.")
else:
    print(f"La colonna '{candidate_columns}' contiene duplicati e non può essere una chiave primaria.")

La colonna '['FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST']' potrebbe essere una chiave primaria.


## Analisi NaN

In [9]:
# Calcola la percentuale di NaN per ciascuna colonna
soglia_nan = 0  # soglia del 50% di valori NaN
colonne_more_0_nan = df.columns[df.isna().mean() > soglia_nan]

# Per ogni colonna che ha più del 50% di NaN, calcola la percentuale di NaN e non NaN
for col in colonne_more_0_nan:
    nan_percent = df[col].isna().mean() * 100  # Percentuale di NaN
    non_nan_percent = 100 - nan_percent  # Percentuale di non NaN
    print(f"Colonna: {col} | NaN: {nan_percent:.2f}% | Non-NaN: {non_nan_percent:.2f}%")


Colonna: DEP_TIME | NaN: 1.54% | Non-NaN: 98.46%
Colonna: DEP_DELAY | NaN: 1.55% | Non-NaN: 98.45%
Colonna: TAXI_OUT | NaN: 1.58% | Non-NaN: 98.42%
Colonna: WHEELS_OFF | NaN: 1.58% | Non-NaN: 98.42%
Colonna: WHEELS_ON | NaN: 1.63% | Non-NaN: 98.37%
Colonna: TAXI_IN | NaN: 1.63% | Non-NaN: 98.37%
Colonna: ARR_TIME | NaN: 1.63% | Non-NaN: 98.37%
Colonna: ARR_DELAY | NaN: 1.84% | Non-NaN: 98.16%
Colonna: CANCELLATION_CODE | NaN: 98.41% | Non-NaN: 1.59%
Colonna: ACTUAL_ELAPSED_TIME | NaN: 1.84% | Non-NaN: 98.16%
Colonna: AIR_TIME | NaN: 1.84% | Non-NaN: 98.16%
Colonna: CARRIER_DELAY | NaN: 81.55% | Non-NaN: 18.45%
Colonna: WEATHER_DELAY | NaN: 81.55% | Non-NaN: 18.45%
Colonna: NAS_DELAY | NaN: 81.55% | Non-NaN: 18.45%
Colonna: SECURITY_DELAY | NaN: 81.55% | Non-NaN: 18.45%
Colonna: LATE_AIRCRAFT_DELAY | NaN: 81.55% | Non-NaN: 18.45%
Colonna: Unnamed: 27 | NaN: 100.00% | Non-NaN: 0.00%


### Analisi delay

In [10]:
# Lista delle colonne *_DELAY che ti interessano (quelle nel tuo output)
delay_columns = [
    'CARRIER_DELAY',
    'WEATHER_DELAY',
    'NAS_DELAY',
    'SECURITY_DELAY',
    'LATE_AIRCRAFT_DELAY'
]

# Ci sono dei valori di Arr_Delay che hanno i valori di delay specifici settati a NAN
# Filtra le righe che hanno almeno un valore x_delay non NaN
delay_rows_non_nan = df.dropna(subset=delay_columns)

# Verifica se ci sono righe dove almeno un *_DELAY è non NaN e altre colonne sono NaN
mask_non_nan = delay_rows_non_nan[delay_columns].notna().all(axis=1)  # Righe con tutti i *_DELAY non NaN

print("Se un delay specifico è non nan allora tutti gli altri pure non lo saranno:" + str(mask_non_nan.all()))

# Trova le righe che non soddisfano il criterio
inconsistent_rows = delay_rows_non_nan[~mask_non_nan]

# Stampa le righe con inconsistenze
if not inconsistent_rows.empty:
    # nel caso un tipo di delay specifico è NaN anche tutti gli altri tipi specifici devono essere NaN
    for index in inconsistent_rows.index:
        print(f"Riga {index} ha NaN in una colonna *_DELAY mentre altre sono non NaN.")
else:
    # Alcuni valori di ARR_DELAY sono dati dalla somma degli altri valori delay
    condition = (delay_rows_non_nan[delay_columns].sum(axis=1) == delay_rows_non_nan['ARR_DELAY'])
    df_invalid = delay_rows_non_nan[~condition]

    if df_invalid.empty:
        print("Non ci sono inconsistenze nelle colonne di DELAY con i valori di DELAY specifi non NaN.")
        print("I valori di ARR_DELAY sono dati dalla somma degli altri valori delay specifi nel caso questi fossero non NaN")
    else:
        print("Ci sono inconsistenze")
        print(df_invalid[['ARR_DELAY'] + delay_columns])



delay_rows_non_nan[['ARR_DELAY'] + delay_columns ]


Se un delay specifico è non nan allora tutti gli altri pure non lo saranno:True
Non ci sono inconsistenze nelle colonne di DELAY con i valori di DELAY specifi non NaN.
I valori di ARR_DELAY sono dati dalla somma degli altri valori delay specifi nel caso questi fossero non NaN


,ARR_DELAY,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
10,60.0,60.0,0.0,0.0,0.0,0.0
32,37.0,25.0,0.0,12.0,0.0,0.0
34,30.0,0.0,0.0,30.0,0.0,0.0
35,33.0,0.0,0.0,0.0,0.0,33.0
38,36.0,36.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
119230,21.0,4.0,0.0,0.0,0.0,17.0
119241,21.0,7.0,0.0,9.0,0.0,5.0
119246,40.0,11.0,0.0,29.0,0.0,0.0
119264,901.0,862.0,0.0,32.0,0.0,7.0


In [11]:
delay_rows_non_nan[(delay_rows_non_nan['ARR_DELAY'] > 0) & (delay_rows_non_nan[delay_columns].sum(axis=1) == 0)]


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27


In [12]:
delay_rows_with_nan = df[df[delay_columns].isna().any(axis=1)]

# Ci sono riche che hanno ARR
if delay_rows_with_nan[delay_columns].isna().all(axis=1).all():
    # Verifica se ci sono sia valori positivi che negativi in ARR_DELAY
    positive_arr_delay = (delay_rows_with_nan['ARR_DELAY'] > 0).any()
    negative_arr_delay = (delay_rows_with_nan['ARR_DELAY'] < 0).any()

    if positive_arr_delay and negative_arr_delay:
        # ci sono righe che hanno tutti nan in delay specifici ed hanno sia arr_delay positivi che positivi
        print("Ci sono righe che hanno tutti nan in delay specifici ed hanno sia arr_delay negativi che positivi")


Ci sono righe che hanno tutti nan in delay specifici ed hanno sia arr_delay negativi che positivi


In [13]:
# Filtra ed elimina le righe con dep_delay > 0 e valori NaN nelle colonne specificate
df = df[~((df['DEP_DELAY'] > 0) & 
           (df[['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']].isna().any(axis=1)))]

# Imposta a zero i valori delle colonne specificate se dep_delay <= 0
cols_to_zero = ['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']
df.loc[df['DEP_DELAY'] <= 0, cols_to_zero] = 0

print(f"Il dataset contiene {len(df)} elementi")

Il dataset contiene 95763 elementi


In [14]:
# Stampa e elimina le colonne
df = df.drop(columns=['Unnamed: 27'] + delay_columns)

### Analisi su cancelled e diverted

In [15]:
# Calcola la percentuale di 0 e 1 per le colonne "diverted" e "canceled"
percentuali = df[['DIVERTED', 'CANCELLED']].apply(lambda x: x.value_counts(normalize=True) * 100)
print(percentuali)

      DIVERTED  CANCELLED
0.0  99.852762  98.047263
1.0   0.147238   1.952737


In [16]:
# Conta le righe dove CANCELLED è 1.0
count_cancelled = df[df['CANCELLED'] == 1.0].shape[0]

# Conta le righe dove DIVERTED è 1.0 e CANCELLED è diverso da 1.0
count_diverted = df[(df['DIVERTED'] == 1.0)].shape[0]

print("Numero di righe con CANCELLED = 1.0:", count_cancelled)
print("Numero di righe con solo DIVERTED = 1.0:", count_diverted)


Numero di righe con CANCELLED = 1.0: 1870
Numero di righe con solo DIVERTED = 1.0: 141


In [17]:
# Filtra il dataframe per le righe in cui 'CANCELLED' è uguale a 1.0
riga_cancellata = df[df['CANCELLED'] == 1.0]

# Verifica quali colonne contengono valori NaN in quella riga
nan_colonne = riga_cancellata.isna().sum()
nan_colonne = nan_colonne[nan_colonne > 0]

# Stampa le colonne che contengono NaN
print(nan_colonne)

DEP_TIME               1836
DEP_DELAY              1836
TAXI_OUT               1856
WHEELS_OFF             1856
WHEELS_ON              1870
TAXI_IN                1870
ARR_TIME               1870
ARR_DELAY              1870
ACTUAL_ELAPSED_TIME    1870
AIR_TIME               1870
dtype: int64


In [18]:
# Filtra il dataframe per le righe in cui 'CANCELLED' è 1 e 'ARR_DELAY' non è NaN
righe_filtrate = df[(df['CANCELLED'] == 1.0) & (df['ARR_DELAY'].notna())]

# Stampa le righe filtrate
righe_filtrate

# LE COLONNE CHE SOPRA HANNO 1870 SONO TUTTI SETTATI A NAN

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE


In [19]:
# Filtra le righe dove "CANCELLED" è 1 e visualizza solo la colonna "CANCELLATION_CODE"
righe_canceled_1 = df.loc[df['CANCELLED'] == 1, 'CANCELLATION_CODE']


# Verifica se ci sono valori NaN nella colonna
if righe_canceled_1.isna().any():
    print("\nCi sono valori NaN nella colonna 'CANCELLATION_CODE' nelle righe dei voli cancellati, quindi che hanno CANCELED pari a 1.0")
else:
    print("\nTutti i valori nella colonna 'CANCELLATION_CODE' sono non NaN nelle righe dei voli cancellati, quindi che hanno CANCELED pari a 1.0.")

print("\nRighe con canceled = 1:")
righe_canceled_1

# Visualizza la frequenza di ciascun valore nella colonna "CANCELLATION_CODE"
print("\nValori unici nella colonna 'CANCELLATION_CODE' con la loro frequenza:")
righe_canceled_1.value_counts()

# A	Carrier	
# B	Weather	
# C	National Air System	
# D	Security


Tutti i valori nella colonna 'CANCELLATION_CODE' sono non NaN nelle righe dei voli cancellati, quindi che hanno CANCELED pari a 1.0.

Righe con canceled = 1:

Valori unici nella colonna 'CANCELLATION_CODE' con la loro frequenza:


CANCELLATION_CODE
B    878
A    628
C    361
D      3
Name: count, dtype: int64

In [20]:
righe_canceled_2 = df.loc[df['CANCELLED'] == 0, 'CANCELLATION_CODE']

# Verifica se tutti i valori sono NaN
if righe_canceled_2.isna().all():
    print("\nTutti i valori nella colonna 'CANCELLATION_CODE' sono NaN nelle righe dei voli NON cancellati, quindi che hanno CANCELED pari a 0.0.")
else:
    print("\nNon tutti i valori nella colonna 'CANCELLATION_CODE' sono NaN nelle righe dei voli NON cancellati, quindi che hanno CANCELED pari a 0.0.")

print("\nRighe con canceled = 2:")
righe_canceled_2


Tutti i valori nella colonna 'CANCELLATION_CODE' sono NaN nelle righe dei voli NON cancellati, quindi che hanno CANCELED pari a 0.0.

Righe con canceled = 2:


0         NaN
2         NaN
3         NaN
4         NaN
5         NaN
         ... 
119266    NaN
119267    NaN
119268    NaN
119269    NaN
119270    NaN
Name: CANCELLATION_CODE, Length: 93893, dtype: object

In [21]:
# SOSTITUIRE CANCELLATION_CODE CON CANCELLATION_REASON

# Mappa dei codici ai rispettivi motivi di cancellazione
cancellation_reasons = {
    'A': 'carrier',
    'B': 'weather',
    'C': 'national air system',
    'D': 'security'
}

# Trova l'indice della colonna CANCELLATION_CODE
col_index = df.columns.get_loc('CANCELLATION_CODE')

# Inserisce la nuova colonna CANCELLATION_REASON nello stesso punto
df.insert(col_index, 'CANCELLATION_REASON', df['CANCELLATION_CODE'].map(cancellation_reasons))

# Rimozione della colonna originale CANCELLATION_CODE
df = df.drop(columns=['CANCELLATION_CODE'])

In [22]:
df[df['CANCELLED'] == 1]


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_REASON,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
52,2009-01-02,US,1991,PHL,ORD,955.0,NaN,NaN,NaN,NaN,...,1134.0,NaN,NaN,1.0,carrier,0.0,159.0,NaN,NaN,678.0
135,2009-01-04,EV,5547,ATL,BTR,1835.0,NaN,NaN,NaN,NaN,...,1909.0,NaN,NaN,1.0,carrier,0.0,94.0,NaN,NaN,449.0
197,2009-01-06,B6,33,ROC,JFK,1935.0,NaN,NaN,NaN,NaN,...,2101.0,NaN,NaN,1.0,weather,0.0,86.0,NaN,NaN,264.0
234,2009-01-07,WN,3683,SNA,SJC,845.0,NaN,NaN,NaN,NaN,...,1000.0,NaN,NaN,1.0,carrier,0.0,75.0,NaN,NaN,342.0
260,2009-01-08,FL,935,JAX,ATL,1020.0,NaN,NaN,NaN,NaN,...,1129.0,NaN,NaN,1.0,carrier,0.0,69.0,NaN,NaN,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119098,2018-12-27,AA,1189,TPA,DFW,601.0,NaN,NaN,NaN,NaN,...,753.0,NaN,NaN,1.0,weather,0.0,172.0,NaN,NaN,929.0
119100,2018-12-27,AA,2292,DFW,JFK,1830.0,NaN,NaN,NaN,NaN,...,2300.0,NaN,NaN,1.0,carrier,0.0,210.0,NaN,NaN,1391.0
119103,2018-12-27,UA,2135,PHX,DEN,1450.0,NaN,NaN,NaN,NaN,...,1631.0,NaN,NaN,1.0,carrier,0.0,101.0,NaN,NaN,602.0
119104,2018-12-27,YV,5916,DFW,MFE,1724.0,NaN,NaN,NaN,NaN,...,1901.0,NaN,NaN,1.0,national air system,0.0,97.0,NaN,NaN,469.0


In [23]:
# Filtra le righe dove "diverted" è 1
righe_diverted_1 = df[df['DIVERTED'] == 1]

# Stampa o visualizza i risultati
print("Righe con diverted = 1:")
righe_diverted_1.head()

Righe con diverted = 1:


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_REASON,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
1532,2009-02-15,US,1289,PHL,SJU,815.0,815.0,0.0,16.0,831.0,...,1307.0,1541.0,NaN,0.0,NaN,1.0,232.0,NaN,NaN,1576.0
1630,2009-02-18,MQ,3569,DFW,CRP,2125.0,2121.0,-4.0,17.0,2138.0,...,2240.0,1003.0,NaN,0.0,NaN,1.0,75.0,NaN,NaN,354.0
1976,2009-02-27,UA,290,SMF,IAD,720.0,714.0,-6.0,10.0,724.0,...,1506.0,1627.0,NaN,0.0,NaN,1.0,286.0,NaN,NaN,2358.0
2263,2009-03-08,WN,680,MCI,MDW,1155.0,1155.0,0.0,8.0,1203.0,...,1310.0,1531.0,NaN,0.0,NaN,1.0,75.0,NaN,NaN,405.0
4692,2009-05-15,DL,1212,LAX,ATL,735.0,731.0,-4.0,14.0,745.0,...,1450.0,1720.0,NaN,0.0,NaN,1.0,255.0,NaN,NaN,1946.0


In [24]:
# Filtra il dataframe per le righe in cui 'CANCELLED' è uguale a 1.0
riga_cancellata = df[df['DIVERTED'] == 1.0]

# Verifica quali colonne contengono valori NaN in quella riga
nan_colonne = riga_cancellata.isna().sum()
nan_colonne = nan_colonne[nan_colonne > 0]

# Stampa le colonne che contengono NaN
print(nan_colonne)

WHEELS_ON               11
TAXI_IN                 11
ARR_TIME                11
ARR_DELAY              141
CANCELLATION_REASON    141
ACTUAL_ELAPSED_TIME    141
AIR_TIME               141
dtype: int64


In [ ]:
df.fillna(0, inplace=True)